### Importing Libraries 

In [15]:
import numpy as np 
import pandas as pd
import os

In [42]:
import ast
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [4]:
df=pd.read_csv('Production_Playlist.csv', parse_dates=['Added At','Album Date'])
df_country = pd.read_csv('Production_Playlist_Countries.csv')

In [5]:
def optimize_memory(df):
    return (
        df
        .astype({
            'Popularity':'int8',
            'BPM':'int16',
            'Dance':'int8',
            'Energy':'int8',
            'Acoustic':'int8',
            'Instrumental':'int8',
            'Happy':'int8', 
            'Speech':'int8', 
            'Live':'int8', 
            'Loud':'int8', 
            'Time Signature':'int8',
            'Key':'category',
            'Camelot':'category'
        })
        .rename(columns={'Parent Genres':'Parent_Genres',
                         'Album Date':'Album_Date',
                         'Time Signature':'Time_Signature',
                         'Added At':'Added_At',
                         'Album Label':'Record_Label'
                         })
        .drop(columns=['#','Spotify Track Id','Spotify Track Img','Song Preview'])
    )

In [6]:
df.memory_usage(deep=True).sum()

733021

In [7]:
optimize_memory(df).memory_usage(deep=True).sum()

371517

In [8]:
playlist = optimize_memory(df)
playlist['Country'] = df_country['Artist']

In [9]:
def missing_pct(df):
    # Calculate missing value and their percentage for each column
    missing_count_percent = df.isnull().sum() * 100 / df.shape[0]
    df_missing_count_percent = pd.DataFrame(missing_count_percent).round(2)
    df_missing_count_percent = df_missing_count_percent.reset_index().rename(
                    columns={
                            'index':'Column',
                            0:'Missing_Percentage (%)'
                    }
                )
    df_missing_value = df.isnull().sum()
    df_missing_value = df_missing_value.reset_index().rename(
                    columns={
                            'index':'Column',
                            0:'Missing_value_count'
                    }
                )
    # Sort the data frame
    #df_missing = df_missing.sort_values('Missing_Percentage (%)', ascending=False)
    Final = df_missing_value.merge(df_missing_count_percent, how = 'inner', left_on = 'Column', right_on = 'Column')
    Final = Final.sort_values(by = 'Missing_Percentage (%)',ascending = False)
    return Final

missing_pct(playlist)

,Column,Missing_value_count,Missing_Percentage (%)
5,Parent_Genres,69,10.21
4,Genres,59,8.73
0,Song,0,0.00
13,Happy,0,0.00
21,Camelot,0,0.00
20,Record_Label,0,0.00
19,Added_At,0,0.00
18,Time_Signature,0,0.00
17,Key,0,0.00
16,Loud,0,0.00


### Basic Overview

In [10]:
playlist[playlist[['Song','Artist']].duplicated(keep='first')].count()

Song              2
Artist            2
Popularity        2
BPM               2
Genres            2
Parent_Genres     2
Album             2
Album_Date        2
Time              2
Dance             2
Energy            2
Acoustic          2
Instrumental      2
Happy             2
Speech            2
Live              2
Loud              2
Key               2
Time_Signature    2
Added_At          2
Record_Label      2
Camelot           2
Country           2
dtype: int64

In [11]:
playlist.nunique()

Song              670
Artist            526
Popularity         86
BPM               117
Genres            457
Parent_Genres     107
Album             617
Album_Date        491
Time              246
Dance              79
Energy             80
Acoustic           93
Instrumental       86
Happy              92
Speech              8
Live                9
Loud               17
Key                24
Time_Signature      4
Added_At          305
Record_Label      439
Camelot            24
Country           527
dtype: int64

### Genre Analysis 

In [12]:
playlist['Genre'] = playlist['Genres'].str.split(",").str[0]
playlist['Parent_Genre'] = playlist['Parent_Genres'].str.split(",").str[0]

playlist[['Artist','Song','Parent_Genres','Parent_Genre','Genres','Genre']].sample(3)

,Artist,Song,Parent_Genres,Parent_Genre,Genres,Genre
131,Biyo,Bliss,NaN,NaN,NaN,NaN
180,Red Hot Chili Peppers,Californication,"Rock, Metal",Rock,"alternative rock, funk metal, funk rock, perma...",alternative rock
628,Skee-Lo,I Wish,Hip Hop,Hip Hop,"hip hop, rap, west coast rap",hip hop


In [13]:
t = playlist.groupby([playlist['Added_At'].dt.year, 'Parent_Genre'])['Song'].count().reset_index().query("Song > 3")

px.bar(x=t.Added_At, y=t.Song, color=t.Parent_Genre).update_layout(
                  title_text="Parent Genre Distribution by Year",
                  title_x=0.5,
                  yaxis_title='Genre types',
                  xaxis_title='Track Added Year',
                  paper_bgcolor="LightSteelBlue",
                  height=600,
                  margin=dict(l=100, r=150, t=100, b=20)
)

In [20]:
playlist[(playlist.Genres.str.contains("indie pop", na=False))].sample(3)

,Song,Artist,Popularity,BPM,Genres,Parent_Genres,Album,Album_Date,Time,Dance,...,Live,Loud,Key,Time_Signature,Added_At,Record_Label,Camelot,Country,Genre,Parent_Genre
304,Polly,Dora Jar,43,90,indie pop,Rock,Digital Meadow,2021-05-28,2:42,75,...,10,-6,D Major,4,2021-12-08,Original Sin,10B,{'Dora Jar': 'N/A'},indie pop,Rock
204,Lisa Baby,WALK THE MOON,32,93,"dance rock, indie poptimism, modern alternativ...","Rock, Pop",Walk The Moon,2012-06-19,3:52,51,...,20,-7,F Major,4,2021-10-03,RCA Records Label,7B,{'WALK THE MOON': 'U.S.'},dance rock,Rock
11,Digital Witness,St. Vincent,38,105,"art pop, chamber psych, electropop, etherpop, ...","Pop, Folk/Acoustic, Rock, Dance/Electronic",St. Vincent,2014-02-25,3:22,78,...,0,-7,F Major,4,2019-01-22,Loma Vista,7B,{'St. Vincent': 'U.S.'},art pop,Pop


In [22]:
playlist[(playlist.Genres.str.contains("house", na=False))].sample(3)

,Song,Artist,Popularity,BPM,Genres,Parent_Genres,Album,Album_Date,Time,Dance,...,Live,Loud,Key,Time_Signature,Added_At,Record_Label,Camelot,Country,Genre,Parent_Genre
428,No Tears - Original Mix,HNNY,16,119,"deep house, electronica, float house, indie ja...","Dance/Electronic, Rock, R&B",Tears,2013-12-20,4:35,95,...,10,-12,A#/B♭ Minor,4,2022-05-22,Local Talk,3A,{'HNNY': 'Sweden'},deep house,Dance/Electronic
622,Eyes on Fire - Zeds Dead Remix,"Blue Foundation,Zeds Dead",54,145,"dream pop, electronica, brostep, canadian elec...","Dance/Electronic, Rock",Eyes on Fire (Zeds Dead Remix) [Zed Dead Remix],2007-01-01,5:20,61,...,10,-6,B Major,4,2022-12-22,WM Denmark,1B,"{'Blue Foundation': 'N/A', 'Zeds Dead': 'Canada'}",dream pop,Dance/Electronic
101,Love Control,"Max Chapman,Teeja",19,126,"deep disco house, house, uk tech house,",Dance/Electronic,Love Control EP,2020-04-24,6:13,81,...,0,-8,F#/G♭ Minor,4,2021-01-08,Resonance Records,11A,"{'Max Chapman': 'N/A', 'Teeja': 'N/A'}",deep disco house,Dance/Electronic


### Instrumental Analysis 

In [23]:
d1, d2 = playlist.Album_Date.dt.strftime("%Y").value_counts(), playlist[playlist.Instrumental==0].Album_Date.dt.strftime("%Y").value_counts()
vals = []
for k in d1.keys():
    if k in d2.keys():
        vals.append(d2[k])
    else:
        vals.append(0)
d1keys = d1.keys()
t = pd.DataFrame({'Year':d1keys,'All':d1.values,'instrumentalzero':vals})

fig = go.Figure()

fig.add_trace(go.Bar(name='All',x=t.Year, y=t.All, offsetgroup=0))
fig.add_trace(go.Bar(name='Instrumental 0',x=t.Year, y=t.instrumentalzero, offsetgroup=0, text=t.instrumentalzero, marker_color = '#204887'))
fig.update_layout(title_text="All Tracks vs Tracks that are 0 Instrumental",
                  yaxis_title='# of tracks',
                  xaxis_title='Album Release Year',
                  paper_bgcolor="LightSteelBlue",
                  margin=dict(l=100, r=150, t=100, b=20)
)

In [24]:
d1, d2 = playlist.Added_At.dt.strftime("%Y").value_counts(), playlist[playlist.Instrumental==0].Added_At.dt.strftime("%Y").value_counts()
vals = []
for k in d1.keys():
    if k in d2.keys():
        vals.append(d2[k])
    else:
        vals.append(0)
d1keys = d1.keys()
t = pd.DataFrame({'Year':d1keys,'All':d1.values,'instrumentalzero':vals}).sort_values('Year').query("Year != '2018'")
fig = go.Figure()

fig.add_trace(go.Bar(name='All',x=t.Year, y=t.All, offsetgroup=0))
fig.add_trace(go.Bar(name='Instrumental 0',x=t.Year, y=t.instrumentalzero, offsetgroup=0, text=t.instrumentalzero, marker_color = '#204887'))
fig.update_layout(title_text="Total Tracks vs Tracks that are 0 Instrumental",
                  yaxis_title='# of tracks',
                  xaxis_title='Year track was added to playlist',
                  paper_bgcolor="LightSteelBlue",
                  margin=dict(l=100, r=150, t=100, b=20)
)

In [25]:
t = (playlist
[playlist.Album_Date.dt.year.isin([2017,2018,2019,2020,2021,2022])]
 .assign(release_year=playlist.Album_Date.dt.year,
         added_year=playlist.Added_At.dt.year)
 .groupby(['release_year','added_year'])
 .size()
 .rename('song_count')
 .reset_index()
)

data = []
for year in t['release_year'].unique():
    df_year = t[t['release_year'] == year]
    data.append(go.Bar(x=df_year['added_year'], y=df_year['song_count'], name=str(year), text=df_year['song_count']))
layout = go.Layout(title='Track Count by Release Year and Added Year', 
                   xaxis_title='Added Year', title_x=0.5,
                   yaxis_title='Track Count',
                   paper_bgcolor="LightSteelBlue",
                   margin=dict(l=100, r=150, t=100, b=20)
            )
fig = go.Figure(data=data, layout=layout)

fig.show()

### Record Labels 

In [26]:
playlist['Record_Label'].nunique()

439

In [28]:
recLabels = playlist['Record_Label'].value_counts().reset_index().query('Record_Label > 5')['index'].tolist()
t = (playlist[playlist['Record_Label'].isin(recLabels)]
 .assign(Added_Year=playlist.Added_At.dt.strftime("%Y"))
 [['Record_Label','Added_Year','Song']]
 .groupby(['Added_Year','Record_Label'])['Song']
 .count().reset_index()
 )

px.bar(x=t.Added_Year, y=t.Song, color=t.Record_Label).update_xaxes(type='category').update_layout(
                  title_text="Record Label distribution by year",
                  title_x=0.5,
                  yaxis_title='Record Labels',
                  xaxis_title='Track Added Year',
                  paper_bgcolor="LightSteelBlue",
                  height=600,
                  margin=dict(l=100, r=150, t=100, b=20)
)

In [29]:
playlist[playlist.Record_Label.str.contains("Columbia")].sample(3)

,Song,Artist,Popularity,BPM,Genres,Parent_Genres,Album,Album_Date,Time,Dance,...,Live,Loud,Key,Time_Signature,Added_At,Record_Label,Camelot,Country,Genre,Parent_Genre
631,Missin You Crazy,Russ,66,132,"hawaiian hip hop, rap",Hip Hop,ZOO,2018-09-07,3:46,72,...,0,-6,F Major,4,2022-12-29,Columbia,7B,{'Russ': 'U.S.'},hawaiian hip hop,Hip Hop
167,Con Altura,"ROSALÍA,J Balvin,El Guincho",71,98,"r&b en espanol, reggaeton, reggaeton colombian...","Latin, R&B",Con Altura,2019-03-28,2:41,88,...,0,-4,F#/G♭ Major,4,2021-06-20,Columbia,2B,"{'ROSALÍA': 'N/A', 'J Balvin': 'Colombia', 'El...",r&b en espanol,Latin
570,Cuando Te Encontré,Matisse,47,94,"latin arena pop, latin pop, mexican pop, urban...",Latin,Sube (Summer Edition),2016-07-08,3:08,67,...,20,-5,D Major,4,2022-10-22,Columbia,10B,{'Matisse': 'France'},latin arena pop,Latin


In [30]:
recLabels = playlist['Record_Label'].value_counts().reset_index().query('Record_Label > 3')['index'].tolist()
(playlist[playlist['Record_Label'].isin(recLabels)]
 .groupby('Record_Label').agg({'Album_Date':['min','max']}))

Album_Date           
                                         min        max
Record_Label                                           
A&M                               2003-06-24 2003-06-24
Atlantic Records                  2018-02-16 2020-08-14
Canvasback/ATL                    2011-09-02 2021-10-27
Capitol Records                   2004-01-01 2022-08-19
Columbia                          1999-11-04 2022-07-01
Domino Recording Co               2006-02-21 2018-08-24
Fueled By Ramen                   2012-02-21 2017-05-12
Geffen                            1991-09-26 2008-01-01
Glassnote Entertainment Group LLC 2009-05-25 2022-09-07
Island Records                    1995-01-01 2022-05-13
Jagjaguwar                        2013-02-05 2021-08-04
Matador                           2008-12-09 2022-06-29
Mom+Pop                           2012-11-09 2019-11-12
Motown                            2007-08-21 2009-01-01
Ninja Tune                        2010-09-20 2020-05-26
Numero Group                      1998-02-24 2006-01-01
PIV Records                       2019-03-08 2021-07-30
RCA Records Label                 1999-11-02 2018-08-17
Rancho Humilde                    2019-10-31 2019-12-24
Sony Music Latin                  2015-06-02 2021-02-03
Stones Throw Records              2019-02-12 2022-06-29
Sumerian Records                  2007-10-07 2010-04-06
Universal-Island Records Ltd.     2002-01-01 2022-06-17
Verzache Music, Inc.              2020-11-13 2021-03-05
Virgin Records                    1993-01-01 2006-01-01
WEA Latina                        2020-09-02 2021-05-28
Warner Records                    1986-03-31 2022-05-02

### BPM & Key Analysis 

In [33]:
playlist['BPM'].describe()

count    676.000000
mean     120.544379
std       27.534374
min       63.000000
25%      100.000000
50%      119.000000
75%      132.000000
max      204.000000
Name: BPM, dtype: float64

In [34]:
playlist['BPM'].median()

119.0

In [35]:
px.bar(playlist.BPM.value_counts()).update_layout(
                  title_text="Tempo of songs", title_x=0.5,
                  yaxis_title='Track Count',
                  xaxis_title='BPM',
                  paper_bgcolor="LightSteelBlue",
                  margin=dict(l=100, r=150, t=100, b=20)
                )

In [36]:
playlist[playlist.BPM==120].sample(3)

,Song,Artist,Popularity,BPM,Genres,Parent_Genres,Album,Album_Date,Time,Dance,...,Live,Loud,Key,Time_Signature,Added_At,Record_Label,Camelot,Country,Genre,Parent_Genre
467,Dancing in My Room,347aidan,69,120,sad rap,Hip Hop,Dancing in My Room,2020-10-26,3:00,98,...,10,-11,G#/A♭ Minor,4,2022-07-02,Columbia,1A,{'347aidan': 'N/A'},sad rap,Hip Hop
559,Too Big - Atjazz Remix,"Kosmo Kint,Atjazz",15,120,"afro house, broken beat, deep house, soulful h...",Dance/Electronic,Too Big (Atjazz Remix),2022-09-19,4:08,80,...,0,-10,G Minor,4,2022-10-02,Toy Tonics,6A,"{'Kosmo Kint': 'N/A', 'Atjazz': 'United Kingdom'}",afro house,Dance/Electronic
250,Trans Love Vibration (Eris Goes to Church),Eris Drew,33,120,"experimental house, float house","New age, Dance/Electronic",Devotion,2018-10-26,5:04,80,...,0,-11,C Major,4,2021-10-24,Naive,8B,{'Eris Drew': 'N/A'},experimental house,New age


In [37]:
playlist[playlist.BPM==120][['Artist','Song','BPM','Parent_Genre','Genre']].sample(3)

,Artist,Song,BPM,Parent_Genre,Genre
633,"Fabich,Pastel,Jafunk,Bambie",Ecstasy,120,R&B,indie soul
329,"Vladimir Cauchemar,Asdek,Laylow,Rim'K",Brrr,120,Hip Hop,french hip hop
516,Tyler Burkhart,Just how I love you,120,Rock,indie pop


In [38]:
playlist.groupby(['BPM','Key']).size().rename('Count').reset_index().query('Count > 3')

,BPM,Key,Count
749,100,B Minor,4
764,100,G Major,4
1230,120,C Major,4
1239,120,E Minor,4
1244,120,G Major,4
1290,122,F#/G♭ Major,4
1316,123,G Major,4


In [39]:
px.bar(playlist['Key'].value_counts()).update_layout(
                  title_text="Song Key",
                  yaxis_title='Track Count',
                  xaxis_title='Key',
                  paper_bgcolor="LightSteelBlue",
                  margin=dict(l=100, r=150, t=100, b=20)
                )

### Spotify Features Analysis 

In [40]:
scatter = go.Scatter(
    x=playlist.Dance,
    y=playlist.Energy,
    mode='markers',
    marker_color=playlist.Happy,
    marker=dict(
        colorscale='teal',
        showscale=True,
        line_width=1,
        size=10,
        colorbar={'title': 'Happy'}
    ),
    hovertext=['Artist: {}<br>Song: {}<br>Dance: {}<br>Energy: {}<br>Happy: {}'\
               .format(a, s, d, e, h)\
                for a ,s, d, e, h in zip(playlist.Artist, playlist.Song, playlist.Dance, playlist.Energy, playlist.Happy)],
)

rect = {
    'type': 'rect',
    'xref': 'x',
    'yref': 'y',
    'x0': playlist.Dance.max()/2,
    'y0': playlist.Energy.max()/2,
    'x1': playlist.Dance.max()+3,
    'y1': playlist.Energy.max()+5,
    'line': {'color': '#0b5685', 'width': 2},
    'opacity': 0.5,
    'fillcolor': 'rgba(250, 220, 60, 0.1)',
}

text = {
    'x': playlist.Dance.max() * 0.95,
    'y': playlist.Energy.max(),
    'text':  'Happy Quadrant',
    'showarrow': False,
    'font': {
        'size': 30,
        'color': 'black',
        'family': 'Balto',
    }
}

layout = go.Layout(shapes=[rect], annotations=[text])

fig = go.Figure(data=[scatter], layout=layout)

fig.update_layout(
                  title_text="Energy vs Dance vs Happiness Distribution",title_x=0.5,
                  yaxis_title='Energy',
                  xaxis_title='Dance',
                  plot_bgcolor="#edf5fa",
                  paper_bgcolor="LightSteelBlue",
                  height=600,
                  margin=dict(l=100, r=150, t=80, b=20)
                )
fig.show()

In [41]:
playlist\
    [(playlist.Energy < 30) & (playlist.Dance > 80)]\
    [['Song','Artist','Energy','Dance','Happy']]

,Song,Artist,Energy,Dance,Happy
264,Cristal (feat. BxRod),"Cráneo,Made in M,BxRod",26,82,34
296,No! No! No! No!,Axel Boman,29,94,40
434,Tears - Original Mix,HNNY,28,88,48
667,Kiss,Prince,27,90,74
